In [1]:

import argparse
import datetime
import json
import random
import time
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader, DistributedSampler

import datasets
import util.misc as utils
from datasets import build_dataset, get_coco_api_from_dataset
from engine import evaluate, train_one_epoch
from models import build_model
from tqdm import tqdm

In [2]:
def get_args_parser():
    parser = argparse.ArgumentParser('Set transformer detector', add_help=False)
    parser.add_argument('--lr', default=1e-4, type=float)
    parser.add_argument('--lr_backbone', default=1e-5, type=float)
    parser.add_argument('--batch_size', default=2, type=int)
    parser.add_argument('--weight_decay', default=1e-4, type=float)
    parser.add_argument('--epochs', default=300, type=int)
    parser.add_argument('--lr_drop', default=200, type=int)
    parser.add_argument('--clip_max_norm', default=0.1, type=float,
                        help='gradient clipping max norm')

    # Model parameters
    parser.add_argument('--num_classes', type=int, default=None,
                        help="Number of classes in dataset+1")
    parser.add_argument('--frozen_weights', type=str, default=None,
                        help="Path to the pretrained model. If set, only the mask head will be trained")
    # * Backbone
    parser.add_argument('--backbone', default='resnet50', type=str,
                        help="Name of the convolutional backbone to use")
    parser.add_argument('--dilation', action='store_true',
                        help="If true, we replace stride with dilation in the last convolutional block (DC5)")
    parser.add_argument('--position_embedding', default='sine', type=str, choices=('sine', 'learned'),
                        help="Type of positional embedding to use on top of the image features")

    # * Transformer
    parser.add_argument('--enc_layers', default=6, type=int,
                        help="Number of encoding layers in the transformer")
    parser.add_argument('--dec_layers', default=6, type=int,
                        help="Number of decoding layers in the transformer")
    parser.add_argument('--dim_feedforward', default=2048, type=int,
                        help="Intermediate size of the feedforward layers in the transformer blocks")
    parser.add_argument('--hidden_dim', default=256, type=int,
                        help="Size of the embeddings (dimension of the transformer)")
    parser.add_argument('--dropout', default=0.1, type=float,
                        help="Dropout applied in the transformer")
    parser.add_argument('--nheads', default=8, type=int,
                        help="Number of attention heads inside the transformer's attentions")
    parser.add_argument('--num_queries', default=100, type=int,
                        help="Number of query slots")
    parser.add_argument('--pre_norm', action='store_true')

    
    ####################### @amirhnazerii #######################
    ##### start 03/27/2025
    # * Classification head
    parser.add_argument('--new_layer_dim', default=None, type=int,
                        help="classification head added fc-layer dim")
    
    ##### end 03/27/2025
    
    
    # * Segmentation
    parser.add_argument('--masks', action='store_true',
                        help="Train segmentation head if the flag is provided")

    # Loss
    parser.add_argument('--no_aux_loss', dest='aux_loss', action='store_false',
                        help="Disables auxiliary decoding losses (loss at each layer)")
    parser.add_argument('--inter_class_weight', default=None, type=float,
                    help="Weight for inter-class distance maximization in CenterLoss - amirhnazerii 5/2/2025")

    
    # * Matcher
    parser.add_argument('--set_cost_class', default=1, type=float,
                        help="Class coefficient in the matching cost")
    parser.add_argument('--set_cost_bbox', default=5, type=float,
                        help="L1 box coefficient in the matching cost")
    parser.add_argument('--set_cost_giou', default=2, type=float,
                        help="giou box coefficient in the matching cost")
    # * Loss coefficients
    parser.add_argument('--mask_loss_coef', default=1, type=float)
    parser.add_argument('--dice_loss_coef', default=1, type=float)
    parser.add_argument('--bbox_loss_coef', default=5, type=float)
    parser.add_argument('--giou_loss_coef', default=2, type=float)
    parser.add_argument('--eos_coef', default=0.1, type=float,
                        help="Relative classification weight of the no-object class")

    # dataset parameters
    parser.add_argument('--dataset_file', default='coco')
    parser.add_argument('--coco_path', type=str)
    parser.add_argument('--coco_panoptic_path', type=str)
    parser.add_argument('--remove_difficult', action='store_true')

    parser.add_argument('--output_dir', default='',
                        help='path where to save, empty for no saving')
    parser.add_argument('--device', default='cuda',
                        help='device to use for training / testing')
    parser.add_argument('--seed', default=42, type=int)
    parser.add_argument('--resume', default='', help='resume from checkpoint')
    parser.add_argument('--start_epoch', default=0, type=int, metavar='N',
                        help='start epoch')
    parser.add_argument('--eval', action='store_true')
    parser.add_argument('--num_workers', default=2, type=int)

    # distributed training parameters
    parser.add_argument('--world_size', default=1, type=int,
                        help='number of distributed processes')
    parser.add_argument('--dist_url', default='env://', help='url used to set up distributed training')
    
    
    # robustness param:
    parser.add_argument('--robust', default=False, type=bool,
                        help='nrobust detr training with modified loss function.')
    
    
    return parser

In [3]:
parser = argparse.ArgumentParser('DETR training and evaluation script', parents=[get_args_parser()])
args = parser.parse_args(args=['--resume', 'detr-r50-modifhead-128fc92fc.pth',
                              '--coco_path', '/scratch/anazeri/coco/',
                               '--output_dir' ,'/home/anazeri/detr_finetune/robust-detr-r50-coco-modifhead-128fc92fc-TEMP',
                               "--lr_drop", "7",  
                              '--new_layer_dim', '128',
                               "--inter_class_weight","0",
                              "--robust", "True",
                               "--epochs", "10"
                              ])

args.distributed = False
# --dataset_file "coco" \
    # --coco_path "/scratch/anazeri/coco/" \
    # --output_dir "/home/anazeri/detr_finetune/detr-r50-coco-modifhead-128fc92fc-epoch10" \
    # --resume "detr-r50-modifhead-128fc92fc.pth" \
    # --lr_drop 3 \
    # --backbone "resnet50" \
    # --epochs 5

In [4]:
if args.world_size > 1:
    utils.init_distributed_mode(args)  # Enable distributed mode if running on multiple GPUs
    print("git:\n  {}\n".format(utils.get_sha()))
else:
    args.distributed = False  # Force single GPU mode


print(args)

device = torch.device(args.device)

# fix the seed for reproducibility
seed = args.seed + utils.get_rank()
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

model, criterion, postprocessors = build_model(args)
model.to(device)

model_without_ddp = model

Namespace(lr=0.0001, lr_backbone=1e-05, batch_size=2, weight_decay=0.0001, epochs=10, lr_drop=7, clip_max_norm=0.1, num_classes=None, frozen_weights=None, backbone='resnet50', dilation=False, position_embedding='sine', enc_layers=6, dec_layers=6, dim_feedforward=2048, hidden_dim=256, dropout=0.1, nheads=8, num_queries=100, pre_norm=False, new_layer_dim=128, masks=False, aux_loss=True, inter_class_weight=0.0, set_cost_class=1, set_cost_bbox=5, set_cost_giou=2, mask_loss_coef=1, dice_loss_coef=1, bbox_loss_coef=5, giou_loss_coef=2, eos_coef=0.1, dataset_file='coco', coco_path='/scratch/anazeri/coco/', coco_panoptic_path=None, remove_difficult=False, output_dir='/home/anazeri/detr_finetune/robust-detr-r50-coco-modifhead-128fc92fc-TEMP', device='cuda', seed=42, resume='detr-r50-modifhead-128fc92fc.pth', start_epoch=0, eval=False, num_workers=2, world_size=1, dist_url='env://', robust=True, distributed=False)


/home/anazeri/.conda/envs/Detr_env1/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/anazeri/.conda/envs/Detr_env1/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
####################### @amirhnazerii #######################
##### start 03/28/2025
## Freeze all model parameters except the classification head
for param in model_without_ddp.parameters():
    param.requires_grad = False
for param in model_without_ddp.class_embed.parameters():
    param.requires_grad = True  # Only train the classification head

# Re-define optimizer (only updates classification head parameters)
optimizer = torch.optim.AdamW(model_without_ddp.class_embed.parameters(), lr=1e-4, weight_decay=1e-4)
#     optimizer = torch.optim.AdamW(param_dicts, lr=args.lr,
#                                   weight_decay=args.weight_decay)

n_parameters = sum(p.numel() for p in model_without_ddp.parameters() if p.requires_grad)
print('number of params:', n_parameters)
##### end   



lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, args.lr_drop)

dataset_train = build_dataset(image_set='train', args=args)
dataset_val = build_dataset(image_set='val', args=args)

if args.distributed:
    sampler_train = DistributedSampler(dataset_train)
    sampler_val = DistributedSampler(dataset_val, shuffle=False)
else:
    sampler_train = torch.utils.data.RandomSampler(dataset_train)
    sampler_val = torch.utils.data.SequentialSampler(dataset_val)

batch_sampler_train = torch.utils.data.BatchSampler(
    sampler_train, args.batch_size, drop_last=True)

data_loader_train = DataLoader(dataset_train, batch_sampler=batch_sampler_train,
                               collate_fn=utils.collate_fn, num_workers=args.num_workers)
data_loader_val = DataLoader(dataset_val, args.batch_size, sampler=sampler_val,
                             drop_last=False, collate_fn=utils.collate_fn, num_workers=args.num_workers)






if args.dataset_file == "coco_panoptic":
    # We also evaluate AP during panoptic training, on original coco DS
    coco_val = datasets.coco.build("val", args)
    base_ds = get_coco_api_from_dataset(coco_val)
else:
    base_ds = get_coco_api_from_dataset(dataset_val)

if args.frozen_weights is not None:
    checkpoint = torch.load(args.frozen_weights, map_location='cpu')
    model_without_ddp.detr.load_state_dict(checkpoint['model'])

output_dir = Path(args.output_dir)
if args.resume:
    if args.resume.startswith('https'):
        checkpoint = torch.hub.load_state_dict_from_url(
            args.resume, map_location='cpu', check_hash=True)
    else:
        checkpoint = torch.load(args.resume, map_location='cpu')
    model_without_ddp.load_state_dict(checkpoint['model'], strict=False)  #### Modified by Amir: , strict=False
    if not args.eval and 'optimizer' in checkpoint and 'lr_scheduler' in checkpoint and 'epoch' in checkpoint:
        optimizer.load_state_dict(checkpoint['optimizer'])
        lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
        args.start_epoch = checkpoint['epoch'] + 1


number of params: 44764
loading annotations into memory...
Done (t=10.54s)
creating index...
index created!
loading annotations into memory...
Done (t=0.34s)
creating index...
index created!


In [6]:
if args.eval:
    test_stats, coco_evaluator = evaluate(model, criterion, postprocessors,
                                          data_loader_val, base_ds, device, args.output_dir)
    if args.output_dir:
        utils.save_on_master(coco_evaluator.coco_eval["bbox"].eval, output_dir / "eval.pth")
    

print("Start training")
start_time = time.time()
for epoch in tqdm(range(args.start_epoch, args.epochs)):
    if args.distributed:
        sampler_train.set_epoch(epoch)
    train_stats = train_one_epoch(
        model, criterion, data_loader_train, optimizer, device, epoch,
        args.clip_max_norm)
    lr_scheduler.step()
    if args.output_dir:
        checkpoint_paths = [output_dir / 'checkpoint.pth']
        # extra checkpoint before LR drop and every 100 epochs
        if (epoch + 1) % args.lr_drop == 0 or (epoch + 1) % 1 == 0:                 # original: (epoch + 1) % 100
            checkpoint_paths.append(output_dir / f'checkpoint{epoch:04}.pth')
        for checkpoint_path in checkpoint_paths:
            utils.save_on_master({
                'model': model_without_ddp.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'epoch': epoch,
                'args': args,
            }, checkpoint_path)

    test_stats, coco_evaluator = evaluate(
        model, criterion, postprocessors, data_loader_val, base_ds, device, args.output_dir
    )

    log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                 **{f'test_{k}': v for k, v in test_stats.items()},
                 'epoch': epoch,
                 'n_parameters': n_parameters}

    if args.output_dir and utils.is_main_process():
        with (output_dir / "log.txt").open("a") as f:
            f.write(json.dumps(log_stats) + "\n")

        # for evaluation logs
        if coco_evaluator is not None:
            (output_dir / 'eval').mkdir(exist_ok=True)
            if "bbox" in coco_evaluator.coco_eval:
                filenames = ['latest.pth']
                if epoch % 50 == 0:
                    filenames.append(f'{epoch:03}.pth')
                for name in filenames:
                    torch.save(coco_evaluator.coco_eval["bbox"].eval,
                               output_dir / "eval" / name)

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))


Start training


  0%|                                                                   | 0/10 [00:02<?, ?it/s]


AssertionError: Only 'labels' loss should be used, got boxes

In [2]:
checkpoint0000= torch.load('/home/anazeri/detr_finetune/detr-r50-coco-modifhead-128fc92fc-TEMP/checkpoint0000.pth', 
           map_location = 'cpu')
checkpoint0000['model']['class_embed.0.bias']

tensor([ 0.0493,  0.0418,  0.0334,  0.0225,  0.0373,  0.0177,  0.0307,  0.0117,
         0.0466,  0.0503,  0.0287,  0.0184,  0.0429,  0.0229,  0.0382,  0.0135,
         0.0389,  0.0164,  0.0227,  0.0522,  0.0467,  0.0428,  0.0323,  0.0227,
         0.0495,  0.0035,  0.0387,  0.0611,  0.0107, -0.0036,  0.0350,  0.0375,
         0.0243,  0.0307,  0.0131,  0.0280,  0.0441,  0.0437,  0.0544,  0.0347,
         0.0440,  0.0024,  0.0245,  0.0317,  0.0275,  0.0205,  0.0376,  0.0561,
         0.0469,  0.0524,  0.0473,  0.0299,  0.0579,  0.0337,  0.0321,  0.0401,
         0.0213,  0.0312,  0.0257,  0.0391,  0.0199,  0.0137,  0.0510,  0.0134,
         0.0303,  0.0086,  0.0352,  0.0094,  0.0260,  0.0238,  0.0245,  0.0134,
         0.0176,  0.0412,  0.0205,  0.0382,  0.0400,  0.0614,  0.0424,  0.0338,
         0.0442,  0.0542,  0.0532,  0.0012,  0.0647,  0.0511,  0.0102,  0.0437,
        -0.0213,  0.0152,  0.0064,  0.0065,  0.0279,  0.0163,  0.0637,  0.0505,
         0.0130,  0.0234,  0.0657,  0.04

In [4]:
checkpoint0009= torch.load('/home/anazeri/detr_finetune/detr-r50-coco-modifhead-128fc92fc-TEMP/checkpoint0009.pth', 
           map_location = 'cpu')
checkpoint0009['model']['class_embed.0.bias']

tensor([ 0.0383, -0.0151,  0.0174,  0.0203, -0.0082, -0.0136,  0.0467, -0.0315,
         0.0432,  0.0296,  0.0200, -0.0265,  0.0202, -0.0029,  0.0249, -0.0037,
        -0.0043, -0.0083, -0.0289,  0.0454, -0.0078,  0.0072,  0.0042, -0.0131,
         0.0441,  0.0098, -0.0198,  0.0711, -0.0446, -0.0542,  0.0385,  0.0290,
         0.0085, -0.0139, -0.0312, -0.0041,  0.0351,  0.0458,  0.0585,  0.0162,
         0.0014, -0.0529, -0.0242, -0.0002,  0.0203, -0.0157,  0.0068,  0.0674,
         0.0019,  0.0373,  0.0073,  0.0351,  0.0220,  0.0068, -0.0129,  0.0121,
         0.0170,  0.0086, -0.0027,  0.0061, -0.0133, -0.0092,  0.0280, -0.0384,
         0.0100, -0.0195,  0.0180, -0.0440, -0.0198,  0.0262, -0.0415, -0.0782,
        -0.0258,  0.0234, -0.0325,  0.0171,  0.0358,  0.0410,  0.0462,  0.0115,
         0.0056,  0.0532,  0.0406, -0.0442,  0.0438,  0.0294, -0.0183, -0.0020,
        -0.0376, -0.0297, -0.0507, -0.0444,  0.0179, -0.0309,  0.0382,  0.0063,
         0.0132,  0.0020,  0.0384,  0.04

In [7]:
####################### @amirhnazerii #######################
##### start 03/28/2025
## Freeze all model parameters except the classification head
for param in model_without_ddp.parameters():
    param.requires_grad = False
for param in model_without_ddp.class_embed.parameters():
    param.requires_grad = True  # Only train the classification head

# Re-define optimizer (only updates classification head parameters)
optimizer = torch.optim.AdamW(model_without_ddp.class_embed.parameters(), lr=1e-4, weight_decay=1e-4)
#     optimizer = torch.optim.AdamW(param_dicts, lr=args.lr,
#                                   weight_decay=args.weight_decay)

n_parameters = sum(p.numel() for p in model_without_ddp.parameters() if p.requires_grad)
print('number of params:', n_parameters)
##### end   



lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, args.lr_drop)

dataset_train = build_dataset(image_set='train', args=args)
dataset_val = build_dataset(image_set='val', args=args)

if args.distributed:
    sampler_train = DistributedSampler(dataset_train)
    sampler_val = DistributedSampler(dataset_val, shuffle=False)
else:
    sampler_train = torch.utils.data.RandomSampler(dataset_train)
    sampler_val = torch.utils.data.SequentialSampler(dataset_val)

batch_sampler_train = torch.utils.data.BatchSampler(
    sampler_train, args.batch_size, drop_last=True)

data_loader_train = DataLoader(dataset_train, batch_sampler=batch_sampler_train,
                               collate_fn=utils.collate_fn, num_workers=args.num_workers)
data_loader_val = DataLoader(dataset_val, args.batch_size, sampler=sampler_val,
                             drop_last=False, collate_fn=utils.collate_fn, num_workers=args.num_workers)


###### END of modification 

NameError: name 'model_without_ddp' is not defined

In [4]:
if args.world_size > 1:
    utils.init_distributed_mode(args)  # Enable distributed mode if running on multiple GPUs
    print("git:\n  {}\n".format(utils.get_sha()))
else:
    args.distributed = False  # Force single GPU mode


print(args)

device = torch.device(args.device)

# fix the seed for reproducibility
seed = args.seed + utils.get_rank()
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

model, criterion, postprocessors = build_model(args)
model.to(device)

model_without_ddp = model
# if args.distributed:
#     model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu])
#     model_without_ddp = model.module

# n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
# print('number of params:', n_parameter

Namespace(lr=0.0001, lr_backbone=1e-05, batch_size=2, weight_decay=0.0001, epochs=300, lr_drop=200, clip_max_norm=0.1, num_classes=None, frozen_weights=None, backbone='resnet50', dilation=False, position_embedding='sine', enc_layers=6, dec_layers=6, dim_feedforward=2048, hidden_dim=256, dropout=0.1, nheads=8, num_queries=100, pre_norm=False, new_layer_dim=128, masks=False, aux_loss=True, set_cost_class=1, set_cost_bbox=5, set_cost_giou=2, mask_loss_coef=1, dice_loss_coef=1, bbox_loss_coef=5, giou_loss_coef=2, eos_coef=0.1, dataset_file='coco', coco_path='/scratch/anazeri/coco/', coco_panoptic_path=None, remove_difficult=False, output_dir='', device='cuda', seed=42, resume='detr-r50-modifhead-128fc92fc.pth', start_epoch=0, eval=False, num_workers=2, world_size=1, dist_url='env://', distributed=False)


/home/anazeri/.conda/envs/Detr_env1/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/anazeri/.conda/envs/Detr_env1/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
if args.dataset_file == "coco_panoptic":
    # We also evaluate AP during panoptic training, on original coco DS
    coco_val = datasets.coco.build("val", args)
    base_ds = get_coco_api_from_dataset(coco_val)
else:
    base_ds = get_coco_api_from_dataset(dataset_val)

if args.frozen_weights is not None:
    checkpoint = torch.load(args.frozen_weights, map_location='cpu')
    model_without_ddp.detr.load_state_dict(checkpoint['model'])

output_dir = Path(args.output_dir)
if args.resume:
    if args.resume.startswith('https'):
        checkpoint = torch.hub.load_state_dict_from_url(
            args.resume, map_location='cpu', check_hash=True)
    else:
        checkpoint = torch.load(args.resume, map_location='cpu')
    model_without_ddp.load_state_dict(checkpoint['model'], strict=False)  #### Modified by Amir: , strict=False
    if not args.eval and 'optimizer' in checkpoint and 'lr_scheduler' in checkpoint and 'epoch' in checkpoint:
        optimizer.load_state_dict(checkpoint['optimizer'])
        lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
        args.start_epoch = checkpoint['epoch'] + 1

if args.eval:
    test_stats, coco_evaluator = evaluate(model, criterion, postprocessors,
                                          data_loader_val, base_ds, device, args.output_dir)
    if args.output_dir:
        utils.save_on_master(coco_evaluator.coco_eval["bbox"].eval, output_dir / "eval.pth")
    

In [8]:
model_without_ddp

Modified_DETR(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerDecoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_feature